<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Sudoku-Example" data-toc-modified-id="Sudoku-Example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Sudoku Example</a></span><ul class="toc-item"><li><span><a href="#Create-sudoku" data-toc-modified-id="Create-sudoku-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create sudoku</a></span></li></ul></li></ul></div>

## Sudoku Example

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from typing import List, Tuple
import pyomo.environ as pyo

### Create sudoku

In [2]:
class SudokuDrawer():
    # https://stackoverflow.com/questions/45471152/how-to-create-a-sudoku-puzzle-in-python
    def __init__(self, base=3):
        self.base=base
        self.side = base*base
        self.line0  = self.expandLine("╔═══╤═══╦═══╗")
        self.line1  = self.expandLine("║ . │ . ║ . ║")
        self.line2  = self.expandLine("╟───┼───╫───╢")
        self.line3  = self.expandLine("╠═══╪═══╬═══╣")
        self.line4  = self.expandLine("╚═══╧═══╩═══╝")
        self.symbol = " 1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"        
    
    def expandLine(self, line):
        return line[0]+line[5:9].join([line[1:5]*(self.base-1)]*self.base)+line[9:13]
    
    def draw(self, board):
        nums   = [ [""]+[str(int(n)) for n in row] for row in board ]
        print(self.line0)
        for r in range(1,self.side+1):
            print( "".join(n+s for n,s in zip(nums[r-1],self.line1.split("."))) )
            print([self.line2,self.line3,self.line4][(r%self.side==0)+(r%self.base==0)])
            
    def draw_from_model(self, model: pyo.ConcreteModel):
        board = np.zeros(shape=(self.base**2, self.base**2))
        for key, val in instance.y.items():
            if val.value > 0.5:
                board[key[0]-1, key[1]-1] = key[2]
        self.draw(board)
        
    def draw_from_sparse(self, sparse):
        board = np.zeros(shape=(self.base**2, self.base**2))
        for row, col, val in sparse:
            board[row-1, col-1] = val
        self.draw(board)


def generate_subsquare(base):
    # Subsquare_region
    subsq_to_row_col = {i: [] for i in range(1, 1+base**2)}
    for row in range(1,1+base**2):
        for col in range(1, 1+base**2):
            row0 = ((row-1)//base)*base
            col0 = ((col-1)//base)*base
            subsq_to_row_col[(row0 + col0//base)+1].append((row,col))
    return subsq_to_row_col
    

def sparse_board(sudoku: List[List]):
    sparse_matrix = sparse.csr_matrix(np.matrix(sudoku))
    board = []
    for (row, col), val in sparse_matrix.todok().items():
        board.append((row+1, col+1, val))
    return board    

subsq_to_row_col = generate_subsquare(base=3)

- Each row must only have one occurrence of each number
- Each column must only have one occurrence of each number
- Each of the nine sub squares must only have one occurrence of each number

**Problem Formulation**

Define ROWS, COLS and VALUES (Ints from 1 to 9)

![Sudoku](../docs/CubeSudoku.png)

Define a binary variable as follows:
$$y[r, c, v] = 1 \hspace{1ex} \text{if v is the value in r, c. 0 otherwise}$$
$$\textbf{Constraints}$$
$$\sum_{c \in Cols} y[r, c, v] = 1, \forall r \in Rows, \forall v \in Values\hspace{1ex} \hspace{1ex}(1)$$
$$\sum_{r \in Rows} y[r, c, v] = 1, \forall c \in Cols, \forall v \in Values\hspace{1ex} \hspace{1ex}(2)$$
$$\sum_{r,c \in SubSquares[i]} y[r, c, v] = 1, \forall i \in SubSquares\hspace{1ex} \hspace{1ex}(3)$$
$$\sum_{v \in Values} y[r, c, v] = 1, \forall r \in Rows, \forall c \in Cols\hspace{1ex} \hspace{1ex}(4)$$

**Integer Cut**

This is done to prevent the same solution to appear again

$$ \text{Define 2 Sets: } S_0 \text{ and } S_1:$$
$$S_0: \hspace{1ex} \text{Indices for those variables whose current solution is 0.}$$
$$S_1: \hspace{1ex} \text{Indices for those variables whose current solution is 1.}$$
$$\sum_{r,c,v \in S_0} y[r, c, v] + \sum_{r,c,v \in S_1}(1 - y[r, c, v]) \geq 1 \hspace{1ex}(5)$$

In [3]:
model = pyo.AbstractModel()
# Starting board as internal variable
model.board = pyo.Set()
# Create sets for constraints
model.rows = pyo.RangeSet(1,9)
model.cols = pyo.RangeSet(1,9)
model.subsquares = pyo.RangeSet(1,9)
model.values_ = pyo.RangeSet(1,9)
# Create Y as Binary: row, col, val
model.y = pyo.Var(model.rows, model.cols, model.values_, 
                  within=pyo.Binary)

# This is a feasability problem (Objective doesn't matter which)
model.obj = pyo.Objective(expr=1.0)

# Constraint of row
def row_constraint(model, r, v):
    return sum(model.y[r,c,v] for c in model.cols) == 1
model.row_constraint = pyo.Constraint(
    model.rows, model.values_, 
    rule=row_constraint,
    name="Constraint 1 - One of a val per row")
# Constraint of Column
def col_constraint(model, c, v):
    return sum(model.y[r,c,v] for r in model.rows) == 1
model.col_constraint = pyo.Constraint(
    model.cols, model.values_, 
    rule=col_constraint,
    name="Constraint 2 - One of a val per Column")
# Constraint on SubSquare
def subsquare_constraint(subsq_to_row_col):
    def _sq_constraint(model, s, v):
        return sum(model.y[r,c,v] for (r, c) in subsq_to_row_col[s]) == 1
    return _sq_constraint
model.subsq_constraint = pyo.Constraint(
    model.subsquares, model.values_, 
    rule=subsquare_constraint(subsq_to_row_col),
    name="Constraint 3 - One of a val per subsquare")
# Constraint of Values
def value_constraint(model, r, c):
    return sum(model.y[r,c,v] for v in model.values_) == 1
model.value_constraint = pyo.Constraint(
    model.rows, model.cols, rule=value_constraint, 
    name="Constraint 4 - One val per Cell")

In [4]:
# Fix initial board values
def build_model(model):
    # Fix variables based on the current board
    for (r,c,v) in model.board:
        model.y[r,c,v].fix(1)

# Remove previously seen solutions
def add_integer_cut(model):
    if not hasattr(model, "integer_cuts"):
        model.integer_cuts = pyo.ConstraintList()
    # Add the integer cut, corresponding to the current solution in the model
    # Note that if it is exactly one of the previous solution it would not satisfy the constraint
    # To satisfy the constraint, at least 1 number should be different
    cut_expr = 0.0
    for r in model.rows:
        for c in model.cols:
            for v in model.values_:
                if not model.y[r,c,v].fixed:
                    # Note, it may not be exactly 1 (Precision error)
                    if model.y[r,c,v].value >= 0.5:
                        cut_expr += (1.0 - model.y[r,c,v])
                    else:
                        cut_expr += model.y[r,c,v]
    model.integer_cuts.add(cut_expr >= 1)

In [5]:
visual_boards = [
    # Single solution
    [[5, 3, 0, 0, 7, 0, 0, 0, 0],
     [6, 0, 0, 1, 9, 5, 0, 0, 0],
     [0, 9, 8, 0, 0, 0, 0, 6, 0],
     [8, 0, 0, 0, 6, 0, 0, 0, 3],
     [4, 0, 0, 8, 0, 3, 0, 0, 1],
     [7, 0, 0, 0, 2, 0, 0, 0, 6],
     [0, 6, 0, 0, 0, 0, 2, 8, 0],
     [0, 0, 0, 4, 1, 9, 0, 0, 5],
     [0, 0, 0, 0, 8, 0, 0, 7, 9]],
    # Single solution
    [[0, 7, 5, 0, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 3, 0, 0, 0],
     [6, 0, 9, 0, 2, 0, 5, 0, 0],
     [8, 0, 0, 0, 5, 0, 0, 4, 0],
     [0, 0, 0, 2, 6, 4, 0, 0, 0],
     [0, 2, 0, 0, 7, 0, 0, 0, 3],
     [0, 0, 3, 0, 9, 0, 4, 0, 6],
     [0, 0, 0, 6, 0, 0, 0, 0, 8],
     [0, 0, 0, 0, 0, 0, 7, 2, 0]],
    # Hard one
    [[0, 2, 0, 7, 0, 0, 4, 0, 0],
     [0, 1, 5, 0, 6, 0, 0, 0, 9],
     [0, 3, 8, 0, 4, 9, 0, 6, 5],
     [0, 0, 2, 0, 9, 0, 0, 0, 4],
     [5, 0, 0, 0, 0, 0, 0, 0, 1],
     [8, 0, 0, 0, 2, 0, 9, 0, 0],
     [9, 6, 0, 8, 3, 0, 1, 7, 0],
     [2, 0, 0, 0, 1, 0, 5, 9, 0],
     [0, 0, 3, 0, 0, 2, 0, 4, 0]],
    # Multiple Solutions
    [[0, 0, 0, 0, 0, 0, 4, 0, 0],
     [0, 1, 5, 0, 6, 0, 0, 0, 9],
     [0, 3, 8, 0, 4, 9, 0, 6, 5],
     [0, 0, 2, 0, 9, 0, 0, 0, 4],
     [5, 0, 0, 0, 0, 0, 0, 0, 1],
     [8, 0, 0, 0, 2, 0, 9, 0, 0],
     [9, 6, 0, 8, 3, 0, 1, 7, 0],
     [2, 0, 0, 0, 1, 0, 5, 9, 0],
     [0, 0, 3, 0, 0, 2, 0, 4, 0]],
]

In [6]:
drawer = SudokuDrawer()

In [7]:
data = {
    "sudoku0": {
        "board": {None: sparse_board(visual_boards[0])}
    },
    "sudoku1": {
        "board": {None: sparse_board(visual_boards[1])}
    },
    "sudoku2": {
        "board": {None: sparse_board(visual_boards[2])}
    },
    "sudoku3": {
        "board": {None: sparse_board(visual_boards[3])}
    }
}

In [9]:
instance = model.create_instance(namespace="sudoku3", data=data)
build_model(instance)

In [14]:
drawer.draw_from_sparse(instance.board.keys())

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 0 │ 0 │ 0 ║ 0 │ 0 │ 0 ║ 4 │ 0 │ 0 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 0 │ 1 │ 5 ║ 0 │ 6 │ 0 ║ 0 │ 0 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 0 │ 3 │ 8 ║ 0 │ 4 │ 9 ║ 0 │ 6 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 0 │ 0 │ 2 ║ 0 │ 9 │ 0 ║ 0 │ 0 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 0 │ 0 ║ 0 │ 0 │ 0 ║ 0 │ 0 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 0 │ 0 ║ 0 │ 2 │ 0 ║ 9 │ 0 │ 0 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 9 │ 6 │ 0 ║ 8 │ 3 │ 0 ║ 1 │ 7 │ 0 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 0 │ 0 ║ 0 │ 1 │ 0 ║ 5 │ 9 │ 0 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 0 │ 0 │ 3 ║ 0 │ 0 │ 2 ║ 0 │ 4 │ 0 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝


In [10]:
solutions = []
while True:
    with pyo.SolverFactory("glpk") as opt:
        results = opt.solve(instance)
        if results.solver.termination_condition != pyo.TerminationCondition.optimal:
            print("All board solutions have been found")
            break
    add_integer_cut(instance)
    solutions.append(instance.clone())
print(f"Number of solutions: {len(solutions)}")

    solver failure.
    solver failure.
    solver failure.
All board solutions have been found
Number of solutions: 2


In [11]:
for instance in solutions:
    drawer.draw_from_model(instance)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 6 │ 2 │ 9 ║ 3 │ 5 │ 7 ║ 4 │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 1 │ 5 ║ 2 │ 6 │ 8 ║ 7 │ 3 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 3 │ 8 ║ 1 │ 4 │ 9 ║ 2 │ 6 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 7 │ 2 ║ 5 │ 9 │ 1 ║ 6 │ 8 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 9 │ 6 ║ 7 │ 8 │ 4 ║ 3 │ 2 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 4 │ 1 ║ 6 │ 2 │ 3 ║ 9 │ 5 │ 7 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 9 │ 6 │ 4 ║ 8 │ 3 │ 5 ║ 1 │ 7 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 2 │ 8 │ 7 ║ 4 │ 1 │ 6 ║ 5 │ 9 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 5 │ 3 ║ 9 │ 7 │ 2 ║ 8 │ 4 │ 6 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 6 │ 2 │ 9 ║ 7 │ 5 │ 3 ║ 4 │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 1 │ 5 ║ 2 │ 6 │ 8 ║ 7 │ 3 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 3 │ 8 ║ 1 │ 4 │ 9 ║ 2 │ 6 │ 5 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 3 │ 7 │ 2 